In [1]:
# Import des librairies utilisées 

import re  
import unidecode
from bs4 import BeautifulSoup
import requests
from math import *
import pandas as pd
import json
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.express as px
from datetime import datetime
import os 
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.shell import inspect_response

# Etape 1 : Collecter les coordonnées GPS de chaque ville via une API

On utilisera https://nominatim.org/ pour avoir les coordonnées GPS des 35 villes (pas de souscription requise). 

Documentation : https://nominatim.org/release-docs/develop/api/Search/

In [7]:
liste_hotels = pd.read_csv('PATH/table_hotels_scrapy.csv')

In [8]:
# Nombre d'hotels disponibles dans la ville (max 20 - on a choisi de ne retenir que les 20 meilleurs hotels)

liste_villes = pd.DataFrame(liste_hotels.groupby('city').size().reset_index(name = "N_hotels"))
liste_villes

,city,N_hotels
0,aigues-mortes,16
1,aix-provence,20
2,amiens,20
3,annecy,20
4,ariege,20
5,avignon,15
6,bayeux,20
7,bayonne,18
8,besancon,20
9,biarritz,20


In [ ]:
# Initialisation des variables latitude et longitude

liste_villes['lat'] = None
liste_villes['lon'] = None

# Récupération des coordonnées pour chaque ville à l'aide d'une boucle
for i, c in enumerate(liste_villes.city):
    coord = requests.get('https://nominatim.openstreetmap.org/search?q={}+France&format=json'.format(c))
    coord2 = json.loads(coord.text)
    liste_villes['lat'][i] = coord2[0]['lat']
    liste_villes['lon'][i] = coord2[0]['lon']

In [10]:
liste_villes.head()

,city,N_hotels,lat,lon
0,aigues-mortes,16,43.5658225,4.1912837
1,aix-provence,20,43.54625275,5.430773370603349
2,amiens,20,49.8941708,2.2956951
3,annecy,20,45.8992348,6.1288847
4,ariege,20,42.9455368,1.4065544156065486


In [7]:
liste_villes.to_csv('PATH/table_villes_coord.csv', index = False, header=True)

# Etape 2 : Collecter les données météorologiques via une API

On va utiliser le site https://openweathermap.org/appid (l'inscription est obligatoire pour obtenir une apikey gratuite) et https://openweathermap.org/api/one-call-api pour rassembler les informations météorologique des 35 villes. 

In [ ]:
#liste_villes = pd.read_csv('PATH/table_villes_coord.csv')

data_meteo = pd.DataFrame([])
for i in range(len(liste_villes)) :
    meteo = requests.get('https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&exclude=current,minutely,hourly,alerts&appid={}'.format(
        str(liste_villes.lat[i]), 
        str(liste_villes.lon[i]),
        'VOTRE_CLE'))

    weather_data = pd.json_normalize(meteo.json()['daily'])
    weather_data['city'] = liste_villes.city[i]
    data_meteo = data_meteo.append(weather_data,ignore_index=True) 

data_meteo.to_csv('PATH/data_meteo.csv', index = False, header=True)

In [12]:
data_meteo.head()

,dt,sunrise,sunset,moonrise,moonset,moon_phase,pressure,humidity,dew_point,wind_speed,...,temp.max,temp.night,temp.eve,temp.morn,feels_like.day,feels_like.night,feels_like.eve,feels_like.morn,rain,city
0,1661079600,1661057676,1661107117,1661036640,1661094900,0.82,1012,44,288.40,4.01,...,302.84,298.67,302.76,294.63,301.81,298.43,302.30,294.45,NaN,aigues-mortes
1,1661166000,1661144144,1661193421,1661125560,1661184540,0.85,1011,41,287.11,6.49,...,303.96,298.64,303.07,294.98,301.56,298.60,302.86,294.76,NaN,aigues-mortes
2,1661252400,1661230611,1661279723,1661214960,1661273760,0.88,1012,41,287.46,5.22,...,303.50,298.38,302.68,295.05,302.37,298.42,302.75,294.94,NaN,aigues-mortes
3,1661338800,1661317079,1661366025,1661304780,1661362560,0.91,1014,46,288.10,5.52,...,301.73,296.84,298.16,294.93,301.87,297.20,298.44,294.70,NaN,aigues-mortes
4,1661425200,1661403546,1661452325,1661394900,1661450820,0.94,1015,58,290.88,4.68,...,300.23,297.18,298.27,295.32,301.19,297.75,298.74,295.50,NaN,aigues-mortes


On obtient les données météorologiques pour les 7 prochains jours en plus d'aujourd'hui. 

In [19]:
data_meteo['city'].value_counts()

aigues-mortes                8
nimes                        8
le-havre                     8
lille                        8
lyon                         8
marseille                    8
mont-saint-michel            8
montauban                    8
paris                        8
grenoble                     8
rouen                        8
saintes-maries               8
st-malo                      8
strasbourg                   8
toulouse                     8
uzes                         8
la-rochelle                  8
eguisheim                    8
aix-provence                 8
besancon                     8
amiens                       8
annecy                       8
ariege                       8
avignon                      8
bayeux                       8
bayonne                      8
biarritz                     8
dijon                        8
bormes-mimosas               8
carcassonne                  8
cassis                       8
chateau-haut-koenigsbourg    8
colliour

# Etape 3 : Classement des destinations

On va maintenant retraiter cette base de données afin de générer un "score" pour chaque ville et donc un classement des villes selon la météo qu'il va faire dans la semaine qui s'annonce.

On décide (un peu arbitrairement et subjectivement) de baser notre classement météorologique sur 2 variables : La temperature ressentie moyenne sur les prochains jours et le nombre de jours de pluie. 

In [20]:
#data_meteo = pd.read_csv('PATH/data_meteo.csv')

data_meteo = data_meteo.groupby(['city']).agg({'feels_like.day': 'mean', 
                                               'rain' : lambda x: x[x != 0].count()})

data_meteo = data_meteo.sort_values(['rain', 'feels_like.day'], ascending=[True, False])

data_meteo.reset_index(inplace=True)
data_meteo.insert(0, 'ID', data_meteo['city'].str[:5])
data_meteo['rank'] = range(1,len(data_meteo) + 1)

data_meteo.to_csv('PATH/data_meteo.csv', index = False, header=True)